<a href="https://colab.research.google.com/github/pj747/qml-experiments/blob/main/Hybrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install wandb
# !wandb login

In [2]:
!pip install pennylane --upgrade
# !pip install pennylane-qulacs["gpu"] --upgrade

  Using cached https://files.pythonhosted.org/packages/61/ad/d18c5113d7c536c7b4544c2741795af7abde9557ef433c7814aa7d10d845/PennyLane-0.16.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/28/be/3a7241d731ba89063780279a5433f5971c1cf41735b64a9f874b7c3ff995/semantic_version-2.6.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/2e/9f/6742425eba1664edc8d6e93dd4536f4552acacd1a7b3a7ac2871657c969a/autoray-0.2.5-py3-none-any.whl


In [3]:
import pennylane as qml
from pennylane import qnn
from pennylane import numpy as np

In [4]:
import torch

In [5]:
from types import SimpleNamespace
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
dataSet = load_breast_cancer()
X = dataSet.data
Y = dataSet.target
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
Y = Y * 2 - np.ones(len(Y))
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=1)
# y_ = torch.unsqueeze(torch.tensor(Y_train), 1)  # used for one-hot encoded labels
# print(np.shape(y_))
# y_hot = torch.nn.functional.one_hot(torch.tensor(Y_train), num_classes=2)

In [6]:
class QuantumOutput(torch.nn.Module):
    ## convert output from (-1, -1) to (0,0)
    def __init__(self):
        super().__init__()
    
    def forward(self, X):
        return X*0 + 0.5

In [7]:
n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnode(inputs, weights):
    for i in range(0,len(inputs), 2):
        qml.RX(inputs[i], wires=[0])
        qml.RX(inputs[i+1], wires=[1])
    qml.Rot(*weights[0], wires=[1])
    qml.CZ(wires=[0,1])
    # qml.templates.BasicEntanglerLayers(weights, wires=range(n_qubits))
    qml.Rot(*weights[1], wires=[0])
    return qml.expval(qml.PauliZ(wires=[0]))

In [8]:
weight_shapes = {"weights" : (2,3)}
qlayer = qml.qnn.TorchLayer(qnode, weight_shapes)


In [9]:
clayer_1 = torch.nn.Linear(30, 30)
clayer_2 = torch.nn.Linear(2, 2)
softmax = torch.nn.Softmax(dim=1)
layers = [clayer_1, qlayer]
torch.nn.init.uniform_(qlayer.weights, a=0.0, b=0.001)
model = torch.nn.Sequential(*layers)

In [10]:
opt = torch.optim.Adam(model.parameters(), lr=0.01)
loss = torch.nn.MSELoss()
X_train = torch.tensor(X_train, requires_grad=True).float()
Y_train = torch.tensor(Y_train, requires_grad=False).float()
y_hot = y_hot.float()

batch_size = 5
batches = 100 // batch_size

data_loader = torch.utils.data.DataLoader(
    list(zip(X_train, Y_train)), batch_size=5, shuffle=True, drop_last=True
)

epochs = 6

for epoch in range(epochs):

    running_loss = 0

    for xs, ys in data_loader:
        opt.zero_grad()

        loss_evaluated = loss(model(xs), ys)
        loss_evaluated.backward()

        opt.step()

        running_loss += loss_evaluated

    avg_loss = running_loss / batches
    print("Average loss over epoch {}: {:.4f}".format(epoch + 1, avg_loss))



NameError: ignored

In [ ]:
y_pred = model(X_train)
y_pred = y_pred.detach().numpy()  
print(y_pred)
threshold = lambda x: 1 if x > 0 else -1 
vfunc = np.vectorize(threshold)
y_pred = vfunc(y_pred)
print(y_pred)

predictions = y_pred
print(Y_train)
correct = [1 if p == p_true else 0 for p, p_true in zip(predictions, Y_train)]
accuracy = sum(correct) / len(correct)
print(f"Accuracy: {accuracy * 100}%")

In [ ]:
print(y_pred)
print(predictions)

In [ ]:
import math


import tensorflow as tf
import tensorflow.keras as K
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam 
tf.keras.backend.set_floatx('float64')

In [ ]:
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def qnode(inputs, weights_1, weights_2):
    for i in range(0,len(inputs)-1,2): 
        qml.RX(inputs[i], wires=[0])
    qml.CZ(wires=[1,0])
    qml.Rot(*weights_1, wires=0)
    qml.Rot(*weights_2, wires=1)
    return (qml.expval(qml.PauliZ(0)))

In [ ]:
weight_shapes = {"weights_1": 3, "weights_2" : 3}
qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=1)
clayer = tf.keras.layers.Dense(5, input_dim=30, activation='relu')
outLayer = tf.keras.layers.Dense(1, activation="sigmoid")
model = tf.keras.models.Sequential([qlayer])

In [ ]:
dataSet = load_breast_cancer()
X = dataSet.data
Y = dataSet.target
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
Y = Y * 2 - np.ones(len(Y))
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=1)
model.compile(Adam(), loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=1, batch_size=1)

In [ ]:
model.predict(Y_test)

In [ ]:
model.summary()

In [ ]:
import sklearn.datasets
data = sklearn.datasets.make_moons()
X = tf.constant(data[0])
Y = tf.one_hot(data[1], depth=2)

opt = tf.keras.optimizers.SGD(learning_rate=0.5)
model.compile(opt, loss='mae', metrics=['accuracy'])

model.fit(X, Y, epochs=8, batch_size=5)
model.summary()

In [ ]:
def makeModel(config):
    numQubits = config.numQubits
    dev = qml.device("qulacs.simulator", wires=numQubits, shots=1000, gpu=True)
    @qml.qnode(dev)
    def qcircuit(inputs, param):
        for i in range(config.numLayers):
            # norm = np.linalg.norm(inputs)
            # norm = norm if norm !=0 else 1 
            for i in range(0, len(inputs)-numQubits, numQubits):
                for j in range(numQubits):
                    qml.RX(inputs[i+j]*2*math.pi/norm, wires=j)
            if config.fullEntangle:
                for j in range(numQubits):
                    for i in range(j):
                        qml.CZ(wires=[j,i])
            else:
                for j in range(numQubits-1):
                    qml.CZ(wires=[j,j+1])
            for j in range(numQubits):
                qml.Rot(param[j][i][0], param[j][i][1], param[j][i][2], wires = [j])
            
        return [qml.expval(qml.PauliZ(0))]
        
    
    return qcircuit

In [ ]:
config = SimpleNamespace(
        numQubits = 1,
        numLayers = 1,
        dev = "default.qubit",
        batchSize = 4,
        numBatches = 300,
        fullEntangle = True
)
node = makeModel(config)
weight_shapes = {"param": (config.numQubits, config.numLayers, 3)}


In [ ]:
qlayer = qml.qnn.KerasLayer(node, weight_shapes, output_dim=1)

In [ ]:
inputLayer = K.layers.InputLayer(30)
clayer = Dense(30)
model = Sequential([inputLayer, qlayer])
model.compile(Adam(), loss = 'mse', metrics=['accuracy'])
model.summary()

In [ ]:
print(X_train.shape)

In [ ]:
fitting = model.fit(X_train, Y_train, epochs=6, batch_size=config.batchSize, verbose=2)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import make_moons

# Set random seeds
np.random.seed(42)
tf.random.set_seed(42)

X, y = make_moons(n_samples=500, noise=0.1)
y_hot = tf.keras.utils.to_categorical(y, num_classes=2)

In [ ]:
import pennylane as qml

n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnode(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]

In [ ]:
n_layers = 6
weight_shapes = {"weights": (n_layers, n_qubits)}


In [ ]:
qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_qubits)


In [ ]:
clayer_1 = tf.keras.layers.Dense(2)
clayer_2 = tf.keras.layers.Dense(2, activation="softmax")
model = tf.keras.models.Sequential([clayer_1, qlayer, clayer_2])

In [ ]:
opt = tf.keras.optimizers.SGD(learning_rate=0.2)
model.compile(opt, loss="mae", metrics=["accuracy"])

In [ ]:
fitting = model.fit(X, y_hot, epochs=6, batch_size=5, validation_split=0.25, verbose=2)